In [1]:
import numpy as np 
import pandas as pd 
#import matplotlib
#import matplotlib.pyplot as plt
#import seaborn as sns
import time

In [15]:
train = pd.read_csv('./bases/sales_train.csv')
#test = pd.read_csv('./bases/test.csv')
#shop = pd.read_csv('./bases/shops.csv')
items = pd.read_csv('./bases/items.csv')
#it_cat = pd.read_csv('./bases/item_categories.csv')
#sub = pd.read_csv('./bases/sample_submission.csv')

# Data Cleaning

In [6]:
train.describe()

,date_block_num,shop_id,item_id,item_price,item_cnt_day
count,2.935849e+06,2.935849e+06,2.935849e+06,2.935849e+06,2.935849e+06
mean,1.456991e+01,3.300173e+01,1.019723e+04,8.908532e+02,1.242641e+00
std,9.422988e+00,1.622697e+01,6.324297e+03,1.729800e+03,2.618834e+00
min,0.000000e+00,0.000000e+00,0.000000e+00,-1.000000e+00,-2.200000e+01
25%,7.000000e+00,2.200000e+01,4.476000e+03,2.490000e+02,1.000000e+00
50%,1.400000e+01,3.100000e+01,9.343000e+03,3.990000e+02,1.000000e+00
75%,2.300000e+01,4.700000e+01,1.568400e+04,9.990000e+02,1.000000e+00
max,3.300000e+01,5.900000e+01,2.216900e+04,3.079800e+05,2.169000e+03


###  Suppression des duplicates

In [7]:
train.duplicated().sum()

6

In [16]:
print("shape" ,train.shape)
train.drop_duplicates(keep='first', inplace=True)
train.duplicated().sum()

print("new shape" ,train.shape)

shape (2935849, 6)
new shape (2935843, 6)


#### prix négatif ou vente négative

In [17]:
print('shape_0 :',train.shape)
train = train[train.item_cnt_day>=0]
train = train[train.item_price>=0]
print('shape_1 :',train.shape)

shape_0 : (2935843, 6)
shape_1 : (2928486, 6)


In [11]:
train.describe()

,date_block_num,shop_id,item_id,item_price,item_cnt_day
count,2.928486e+06,2.928486e+06,2.928486e+06,2.928486e+06,2.928486e+06
mean,1.456976e+01,3.300294e+01,1.020028e+04,8.894670e+02,1.248338e+00
std,9.422956e+00,1.622543e+01,6.324391e+03,1.727500e+03,2.619589e+00
min,0.000000e+00,0.000000e+00,0.000000e+00,7.000000e-02,1.000000e+00
25%,7.000000e+00,2.200000e+01,4.477000e+03,2.490000e+02,1.000000e+00
50%,1.400000e+01,3.100000e+01,9.355000e+03,3.990000e+02,1.000000e+00
75%,2.300000e+01,4.700000e+01,1.569100e+04,9.990000e+02,1.000000e+00
max,3.300000e+01,5.900000e+01,2.216900e+04,3.079800e+05,2.169000e+03


### Détection des items dont le prix varie fortement dans le mois

In [18]:

X = train.groupby(['date_block_num','item_id']).agg( {"item_price": ["mean","std"]} )
X.columns = ['item_price_mean','item_price_std']
X['coef_var'] = X['item_price_std']/X['item_price_mean']*100
X.reset_index(inplace =True)

X[['date_block_num','item_id','coef_var']].sort_values(by="coef_var",ascending=False)

,date_block_num,item_id,coef_var
45500,5,11863,818.800359
45493,5,11856,702.738019
53915,6,11853,697.297041
57350,6,20141,692.766611
45507,5,11870,667.947193
...,...,...,...
233761,33,21979,NaN
233772,33,22020,NaN
233779,33,22072,NaN
233791,33,22111,NaN


In [19]:
print(X.shape)
X[X.coef_var>20].shape

(233802, 5)


(15055, 5)

Les items par mois ayant une variation de prix de plus de 20% seront supprimés

In [20]:
X = X[['date_block_num','item_id','coef_var']]
train = pd.merge(train, X, on=['date_block_num','item_id'],how="left")
del X
print("train_size_0",train.shape)
train = train[train.coef_var<=20]
train = train.drop(['coef_var'],axis = 1)
print("final_size",train.shape)

train_size_0 (2928486, 7)
final_size (2619916, 6)


## Traitement variable d'intérêt

In [21]:
X_train = train.copy()
X_train = X_train.groupby(['date_block_num', 'shop_id', 'item_id'], as_index=False)['item_cnt_day'].sum()

In [22]:
cols  = ["date_block_num", "shop_id", "item_id"]

group = train.groupby(['date_block_num', 'shop_id', 'item_id']).agg( {"item_cnt_day": ["sum"]} )
group.columns=['item_cnt_month']
group.reset_index( inplace = True)
X_train = X_train.drop(['item_cnt_day'],axis = 1)
X_train = pd.merge( X_train, group, on =cols, how = "left" )

In [23]:
X_train.describe()

,date_block_num,shop_id,item_id,item_cnt_month
count,1.420689e+06,1.420689e+06,1.420689e+06,1.420689e+06
mean,1.481506e+01,3.275791e+01,1.075354e+04,2.310688e+00
std,9.459393e+00,1.647038e+01,6.234353e+03,8.778087e+00
min,0.000000e+00,0.000000e+00,1.000000e+00,1.000000e+00
25%,7.000000e+00,2.100000e+01,5.183000e+03,1.000000e+00
50%,1.400000e+01,3.100000e+01,1.064200e+04,1.000000e+00
75%,2.300000e+01,4.700000e+01,1.608500e+04,2.000000e+00
max,3.300000e+01,5.900000e+01,2.216800e+04,1.644000e+03


In [24]:
X_train = pd.merge(X_train , items.drop(['item_name'],axis =1), on =['item_id'],how="left" )
X_train = X_train.reindex(columns=['date_block_num','shop_id','item_category_id','item_id','item_cnt_month'])
del items
X_train

,date_block_num,shop_id,item_category_id,item_id,item_cnt_month
0,0,0,40,32,6.0
1,0,0,37,33,3.0
2,0,0,40,35,1.0
3,0,0,40,43,1.0
4,0,0,57,51,2.0
...,...,...,...,...,...
1420684,33,59,83,22087,6.0
1420685,33,59,83,22088,2.0
1420686,33,59,83,22091,1.0
1420687,33,59,42,22100,1.0


### Définition de la Fonction qui a tout X(t) associe X(t-lag)

In [25]:
def lag_feature( df,lags, cols ):
    for col in cols:
        tmp = df[["date_block_num", "shop_id","item_id",col ]]
        for i in lags:
            shifted = tmp.copy()
            shifted.columns = ["date_block_num", "shop_id", "item_id", col + "_lag_"+str(i)]
            shifted.date_block_num = shifted.date_block_num + i
            df = pd.merge(df, shifted, on=['date_block_num','shop_id','item_id'], how='left')
        
    return df

In [26]:
X_train = lag_feature( X_train, [1,2,3], ["item_cnt_month"] )
X_train[X_train.date_block_num>2] = X_train[X_train.date_block_num>2].fillna(0)

Dans cette partie, on catégorise les obersations, selon que les ventes au mois t-1 ont été supérieurs ou non à 20. La base est en effet assez déséquilibrée, et dans la suite, plusieurs calculs de moyenne seront faites. La catégorisation représentera la division en groupe homogène.
La catégorisation sera faite avec la variable item_cnt_lag_1 (quantité vendu au mois t-1) , comparativement à la variable item_cnt_month(quantité vendue au mois t).

In [27]:


a = pd.DataFrame((X_train.item_cnt_month>20).values)
a.columns =["cat"]
X_train = X_train.join(a)

a = pd.DataFrame((X_train.item_cnt_month_lag_1>20).values)
a.columns =["cat_lag_1"] # Variable discriminante
X_train = X_train.join(a)

In [28]:
X_train[['cat','cat_lag_1']]
test = (X_train['cat'] == X_train['cat_lag_1'])

test.sum()/X_train.shape[0] *100

99.09297530986726

In [29]:
X_train = X_train[test]
X_train = X_train.drop(['cat'],axis=1)

In [31]:
# Pour un mois , moyenne des quantités mensuelles vendues

group = X_train.groupby( ["date_block_num","cat_lag_1"] ).agg({"item_cnt_month" : ["mean"]})
group.columns = ["date_avg_item_cnt"]
group.reset_index(inplace = True)

X_train = pd.merge(X_train, group, on = ["date_block_num","cat_lag_1"], how = "left")
X_train.date_avg_item_cnt = X_train["date_avg_item_cnt"].astype(np.float16)
X_train = lag_feature( X_train, [1,2,3], ["date_avg_item_cnt"] )
X_train.drop( ["date_avg_item_cnt"], axis = 1, inplace = True )

In [201]:
X_train

,date_block_num,shop_id,item_category_id,item_id,item_cnt_month,item_cnt_month_lag_1,item_cnt_month_lag_2,item_cnt_month_lag_3,cat_lag_1,date_avg_item_cnt_lag_1,date_avg_item_cnt_lag_2,date_avg_item_cnt_lag_3
0,0,0,40,32,6.0,NaN,NaN,NaN,False,NaN,NaN,NaN
1,0,0,37,33,3.0,NaN,NaN,NaN,False,NaN,NaN,NaN
2,0,0,40,35,1.0,NaN,NaN,NaN,False,NaN,NaN,NaN
3,0,0,40,43,1.0,NaN,NaN,NaN,False,NaN,NaN,NaN
4,0,0,57,51,2.0,NaN,NaN,NaN,False,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
1407798,33,59,83,22087,6.0,3.0,2.0,5.0,False,1.75,1.689453,1.708008
1407799,33,59,83,22088,2.0,1.0,7.0,7.0,False,1.75,1.689453,1.708008
1407800,33,59,83,22091,1.0,3.0,0.0,0.0,False,1.75,NaN,NaN
1407801,33,59,42,22100,1.0,1.0,0.0,0.0,False,1.75,NaN,NaN


In [32]:
# Pour un mois et pour un item , moyenne des quantités mensuelles vendues

group = X_train.groupby( ['date_block_num', "cat_lag_1",'item_id'] ).agg({"item_cnt_month" : ["mean"]})
group.columns = ["date_item_avg_item_cnt"]
group.reset_index(inplace = True)

X_train = pd.merge(X_train, group, on = ['date_block_num', "cat_lag_1",'item_id'], how = "left")
X_train.date_item_avg_item_cnt = X_train["date_item_avg_item_cnt"].astype(np.float16)
X_train = lag_feature( X_train, [1,2,3], ["date_item_avg_item_cnt"] )
X_train.drop( ["date_item_avg_item_cnt"], axis = 1, inplace = True )

In [33]:
# Pour un mois et une boutique , moyenne des quantités mensuelles vendues

group = X_train.groupby( ["date_block_num","cat_lag_1","shop_id"] ).agg({"item_cnt_month" : ["mean"]})
group.columns = ["date_shop_avg_item_cnt"]
group.reset_index(inplace = True)

X_train = pd.merge(X_train, group, on = ['date_block_num',"cat_lag_1",'shop_id'], how = "left")
X_train.date_shop_avg_item_cnt = X_train["date_shop_avg_item_cnt"].astype(np.float16)
X_train = lag_feature( X_train, [1,2,3], ["date_shop_avg_item_cnt"] )
X_train.drop( ["date_shop_avg_item_cnt"], axis = 1, inplace = True )

In [34]:
## Pour un mois et une catégorie , moyenne des quantités mensuelles vendues

group = X_train.groupby(['date_block_num',"cat_lag_1",'item_category_id']).agg({"item_cnt_month":['mean']})
group.columns = ['date_category_avg_item_cnt_month']
group.reset_index(inplace=True)
X_train = X_train.merge(group , on =['date_block_num',"cat_lag_1",'item_category_id'] , how = "left")
X_train = lag_feature(X_train , [1,2,3],['date_category_avg_item_cnt_month'])
X_train = X_train.drop(['date_category_avg_item_cnt_month'],axis =1)

In [35]:
## Pour un mois , une boutique et une catégorie , moyenne des quantités mensuelles vendues

group = X_train.groupby(['date_block_num',"cat_lag_1",'shop_id','item_category_id']).agg({"item_cnt_month":['mean']})
group.columns = ['date_shop_category_avg_item_cnt_month']
group.reset_index(inplace=True)
X_train = X_train.merge(group , on =['date_block_num',"cat_lag_1",'shop_id','item_category_id'] , how = "left")
X_train = lag_feature(X_train , [1,2,3],['date_shop_category_avg_item_cnt_month'])
X_train = X_train.drop(['date_shop_category_avg_item_cnt_month'],axis =1)

Les ventes au mois(t-lag) ayant NA sont considérées comme nulle car dans la base X_train , aucune vente mensuelle n'était renseignée à 0

In [36]:
X_train[X_train.date_block_num>2] = X_train[X_train.date_block_num>2].fillna(0)

### Création des variables relatives au prix

In [38]:
# Pour un mois, prix moyen de l'item vendu

group = train.groupby( ["date_block_num","item_id"] ).agg( {"item_price": ["mean"]} )
group.columns = ["date_avg_item_price"]
group.reset_index(inplace = True)
X_train = X_train.merge(group, on = ["date_block_num","item_id"], how = "left")
X_train["date_avg_item_price"] = X_train.date_avg_item_price.astype(np.float16)
lags = [1, 2, 3]
X_train = lag_feature( X_train, lags, ["date_avg_item_price"] )
X_train = X_train.drop(['date_avg_item_price'],axis =1)

In [39]:
# Pour un mois et dans une boutique , prix moyen de l'item vendu

group = train.groupby( ["date_block_num",'shop_id',"item_id"] ).agg( {"item_price": ["mean"]} )
group.columns = ["date_shop_avg_item_price"]
group.reset_index(inplace = True)
X_train = X_train.merge(group, on = ["date_block_num",'shop_id',"item_id"], how = "left")
X_train["date_shop_avg_item_price"] = X_train.date_shop_avg_item_price.astype(np.float16)
lags = [1, 2, 3]
X_train = lag_feature( X_train, lags, ["date_shop_avg_item_price"] )
X_train = X_train.drop(['date_shop_avg_item_price'],axis =1)

In [40]:
del group , cols
X_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1407803 entries, 0 to 1407802
Data columns (total 30 columns):
 #   Column                                       Non-Null Count    Dtype  
---  ------                                       --------------    -----  
 0   date_block_num                               1407803 non-null  int64  
 1   shop_id                                      1407803 non-null  int64  
 2   item_category_id                             1407803 non-null  int64  
 3   item_id                                      1407803 non-null  int64  
 4   item_cnt_month                               1407803 non-null  float64
 5   item_cnt_month_lag_1                         1292401 non-null  float64
 6   item_cnt_month_lag_2                         1273232 non-null  float64
 7   item_cnt_month_lag_3                         1255758 non-null  float64
 8   cat_lag_1                                    1407803 non-null  bool   
 9   date_avg_item_cnt_lag_1                      1

# Imputation des valeurs manquantes

In [41]:
def imputation(df):
    cols = df.select_dtypes(exclude=['object']).columns.tolist()
    
    for col in cols:
        
        if(df[col].isna().sum()!=0):
            moy = df.groupby(['date_block_num','item_id']).agg( {col: ["mean"]})
            moy.columns=['moy col']
            moy.reset_index(inplace=True)
            a = pd.merge(df ,moy , on =['date_block_num','item_id'] , how="left")
            moy = a[['moy col']]
            a = a[[col]]
            moy.columns = a.columns.tolist()
            a=a.combine_first(moy)
            df = df.drop([col],axis =1)
            df = df.join(a)
            
        if(df[col].isna().sum()!=0):
            moy = df.groupby(['date_block_num','item_category_id']).agg( {col: ["mean"]})
            moy.columns=['moy col']
            moy.reset_index(inplace=True)
            a = pd.merge(df ,moy , on =['date_block_num','item_category_id'] , how="left")
            moy = a[['moy col']]
            a = a[[col]]
            moy.columns = a.columns.tolist()
            a=a.combine_first(moy)
            df = df.drop([col],axis =1)
            df = df.join(a)  
            
        if(df[col].isna().sum()!=0):
            moy = df.groupby(['date_block_num','shop_id']).agg( {col: ["mean"]})
            moy.columns=['moy col']
            moy.reset_index(inplace=True)
            a = pd.merge(df ,moy , on =['date_block_num','shop_id'] , how="left")
            moy = a[['moy col']]
            a = a[[col]]
            moy.columns = a.columns.tolist()
            a=a.combine_first(moy)
            df = df.drop([col],axis =1)
            df = df.join(a)

        if(df[col].isna().sum()!=0):
            moy = df.groupby(['date_block_num']).agg( {col: ["mean"]})
            moy.columns=['moy col']
            moy.reset_index(inplace=True)
            a = pd.merge(df ,moy , on =['date_block_num'] , how="left")
            moy = a[['moy col']]
            a = a[[col]]
            moy.columns = a.columns.tolist()
            a=a.combine_first(moy)
            df = df.drop([col],axis =1)
            df = df.join(a)
            
    return df

In [42]:
%%time
X_train = imputation(X_train)

Wall time: 3min 46s


In [43]:
X_train[X_train.date_block_num>2].isna().sum()

date_block_num                                 0
shop_id                                        0
item_category_id                               0
item_id                                        0
item_cnt_month                                 0
cat_lag_1                                      0
item_cnt_month_lag_1                           0
item_cnt_month_lag_2                           0
item_cnt_month_lag_3                           0
date_avg_item_cnt_lag_1                        0
date_avg_item_cnt_lag_2                        0
date_avg_item_cnt_lag_3                        0
date_item_avg_item_cnt_lag_1                   0
date_item_avg_item_cnt_lag_2                   0
date_item_avg_item_cnt_lag_3                   0
date_shop_avg_item_cnt_lag_1                   0
date_shop_avg_item_cnt_lag_2                   0
date_shop_avg_item_cnt_lag_3                   0
date_category_avg_item_cnt_month_lag_1         0
date_category_avg_item_cnt_month_lag_2         0
date_category_avg_it

In [45]:
X_train = X_train[X_train.date_block_num>2]
X_train.to_csv('./bases/Feature_Eng_res.csv', index=False, header=True)